In [1]:
import sys
import pathlib
from itertools import product, tee
from dataclasses import dataclass
from typing import Tuple, List, Optional
import math

In [8]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
import numpy as np
import momepy
from shapely import geometry, ops
from shapely.geometry import LineString
from tqdm.auto import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sys.path.append("..")

In [21]:
from src.preprocess import transform_df_to_trajectory_gdf
from src.visualize import random_colors
from src.trajectory import Trajectory
from src.opendd import extract_samples_from_sqlite

In [48]:
mpl.rcParams["figure.facecolor"] = "w"

In [49]:
FIGURES_PATH = pathlib.Path("../reports/figures")

# Extract reference paths

In [53]:
from src.path_extraction import visualize_traffic_lanes, traffic_lanes_to_graph, \
    extract_paths_from_graph, driving_path_overview_plot

In [56]:
reference_path_lookup = dict()

In [57]:
for roundabout_dir in ROUNDABOUT_DIRS:
    
    roundabout_name = roundabout_dir.name
    print(f"Extracting reference paths from '{roundabout_name}'")
    
    shapefiles_trafficlanes_path = roundabout_dir / f"map_{roundabout_name}/shapefiles_trafficlanes"
    
    trafficlanes = gpd.read_file(shapefiles_trafficlanes_path)
    
    G = traffic_lanes_to_graph(trafficlanes)
    
    paths = extract_paths_from_graph(G)
    
    # save in lookup
    reference_path_lookup[roundabout_name] = paths
    
    fig = driving_path_overview_plot(trafficlanes, paths)
    fig.suptitle(roundabout_name, y=1.01, fontsize=15)
    
    # save images
    roundabout_paths_overview_plot_path = FIGURES_PATH / f"{roundabout_name}_reference_paths.png"
    fig.savefig(roundabout_paths_overview_plot_path, dpi=200)
    plt.close(fig)
    

Extracting reference paths from 'rdb1'
Extracting reference paths from 'rdb2'
Extracting reference paths from 'rdb3'


# Read data

In [33]:
RAW_DATA_PATH = pathlib.Path("../data/raw")
INTERIM_DATA_PATH = pathlib.Path("../data/interim")

In [36]:
ROUNDABOUT_DIRS = list(RAW_DATA_PATH.rglob(r"rdb[1-9]"))

In [35]:
for roundabout_dir in ROUNDABOUT_DIRS:
    
    roundabout_name = roundabout_dir.name
    print(f"Extracting '{roundabout_name}'")
    
    # Extract database
    roundabout_db = list(roundabout_dir.glob("*.sqlite"))[0]    
    df_roundabout = extract_samples_from_sqlite(roundabout_db)
    
    # Save to disk
    output_path_roundabout = INTERIM_DATA_PATH / f"{roundabout_name}.parquet"
    df_roundabout.to_parquet(output_path_roundabout)

Extracting 'rdb1'
Found 153 measurements


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 153/153 [02:14<00:00,  1.13it/s]


Extracting 'rdb2'
Found 56 measurements


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 56/56 [00:32<00:00,  1.71it/s]


Extracting 'rdb3'
Found 54 measurements


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:24<00:00,  2.23it/s]
